# Layer Normalization Experiment - Testing if LayerNorm Prevents Saturation

**Hypothesis:** LayerNorm normalizes pre-activations to ~0 mean, ~1 std before tanh, keeping outputs in the linear region where gradients flow properly.

## Architecture

| Network | Hidden Activations | Pre-Output | Output Activation | Architecture |
|---------|-------------------|------------|-------------------|---------------|
| Actor   | ReLU | **LayerNorm** | tanh | 7->64->32->LN->tanh->3 |
| Critic  | ReLU | None | linear | 510->512->128->1 |

## Learning Rate Order (High to Low)
Experiments run from **HIGH to LOW** learning rates to prioritize early-stopping cases:
- Actor LRs: `[0.1, 0.01, 0.001, 0.0001]`
- Critic LRs: `[0.1, 0.01, 0.001, 0.0001]`
- Total: **16 experiments**

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

## Cell 2: Upload and Extract Experiment Files

**Option A:** Upload `layernorm_experiment.zip` when prompted

**Option B:** If you already uploaded to Drive, skip the upload prompt

In [ ]:
import os
import zipfile

# Check if files already exist
if os.path.exists('/content/layernorm_experiment/mec_env.py'):
    print("Experiment files already present!")
else:
    # Try to find zip in Drive first
    drive_zip = '/content/drive/MyDrive/layernorm_experiment.zip'
    
    if os.path.exists(drive_zip):
        print(f"Found zip in Drive: {drive_zip}")
        zip_path = drive_zip
    else:
        # Upload zip file
        print("Upload layernorm_experiment.zip:")
        from google.colab import files
        uploaded = files.upload()
        zip_path = list(uploaded.keys())[0]
    
    # Extract
    print(f"Extracting {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall('/content')
    
    print("\nExtracted files:")
    for item in os.listdir('/content'):
        if not item.startswith('.'):
            print(f"  {item}")

## Cell 3: Check GPU and Environment

In [ ]:
import torch
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("")
    print("WARNING: No GPU detected!")
    print("Go to: Runtime -> Change runtime type -> GPU")

# Set working directory
os.chdir('/content')
print(f"\nWorking directory: {os.getcwd()}")

## Cell 4: Verify LayerNorm Architecture

In [ ]:
import sys
sys.path.insert(0, '/content/layernorm_experiment')

from Model import LayerNormActorNetwork, CriticNetwork, ActorNetwork
import torch

# Compare architectures
original_actor = ActorNetwork(7, 3, torch.tanh)
layernorm_actor = LayerNormActorNetwork(7, 3, torch.tanh)
critic = CriticNetwork(350, 150, 7, 3)

print("=" * 60)
print("ARCHITECTURE COMPARISON")
print("=" * 60)
print(f"\nOriginal Actor: 7 -> 64 -> 32 -> tanh -> 3")
print(f"  Parameters: {sum(p.numel() for p in original_actor.parameters()):,}")
print(f"\nLayerNorm Actor: 7 -> 64 -> 32 -> LayerNorm -> tanh -> 3")
print(f"  Parameters: {sum(p.numel() for p in layernorm_actor.parameters()):,}")
print(f"\nCritic: 510 -> 512 -> 128 -> 1 (linear)")
print(f"  Parameters: {sum(p.numel() for p in critic.parameters()):,}")

# Show the LayerNorm layer
print("\n" + "=" * 60)
print("LayerNorm Actor Modules:")
print("=" * 60)
for name, module in layernorm_actor.named_modules():
    if name:
        print(f"  {name}: {module}")

## Cell 5: Run All 16 Experiments

This will run all 16 experiments with **HIGH learning rates FIRST**:
- Actor LRs: `[0.1, 0.01, 0.001, 0.0001]` (high to low)
- Critic LRs: `[0.1, 0.01, 0.001, 0.0001]` (high to low)

**Progress is auto-saved to Google Drive every 100 episodes.**

In [ ]:
import sys
sys.path.insert(0, '/content')
sys.path.insert(0, '/content/layernorm_experiment')

from run_layernorm_experiment import run_all_experiments

# Run all experiments
run_all_experiments()

## Cell 6: Check Experiment Status

In [ ]:
import json
import os

results_dir = '/content/results/layernorm_experiment'
status_file = os.path.join(results_dir, 'experiment_status.json')

if os.path.exists(status_file):
    with open(status_file) as f:
        status = json.load(f)
    print("Experiment Status:")
    print(f"  Completed: {len(status['completed'])}/16")
    if status['in_progress']:
        print(f"  In progress: {status['in_progress']}")
    print("\nCompleted experiments:")
    for exp in sorted(status['completed']):
        print(f"  - {exp}")
else:
    print("No status file found yet.")

# Also check Drive backup
drive_dir = '/content/drive/MyDrive/layernorm_results'
if os.path.exists(drive_dir):
    print(f"\nDrive backup exists: {drive_dir}")
    contents = os.listdir(drive_dir)
    print(f"  Contents: {contents[:5]}..." if len(contents) > 5 else f"  Contents: {contents}")

## Cell 7: View Results Summary

In [ ]:
import json
import os

results_dir = '/content/results/layernorm_experiment'

if os.path.exists(results_dir):
    print("LayerNorm Experiment Results Summary:")
    print("="*70)
    print(f"{'Actor LR':<12} {'Critic LR':<12} {'Stop Ep.':<12} {'Final Reward':<15}")
    print("-"*70)
    
    results = []
    for exp_dir in sorted(os.listdir(results_dir)):
        result_file = os.path.join(results_dir, exp_dir, 'results.json')
        if os.path.exists(result_file):
            with open(result_file) as f:
                data = json.load(f)
            results.append(data)
    
    for data in sorted(results, key=lambda x: (-x['actor_lr'], -x['critic_lr'])):
        print(f"{data['actor_lr']:<12} {data['critic_lr']:<12} {data['stopping_episode']:<12} {data['final_reward']:<15.4f}")
    print("="*70)
else:
    print("No results directory found yet.")

## Cell 8: Compare with Original (No LayerNorm) Experiment

In [ ]:
import json
import os

def load_results(results_dir):
    results = []
    if os.path.exists(results_dir):
        for exp_dir in os.listdir(results_dir):
            result_file = os.path.join(results_dir, exp_dir, 'results.json')
            if os.path.exists(result_file):
                with open(result_file) as f:
                    results.append(json.load(f))
    return results

layernorm_results = load_results('/content/results/layernorm_experiment')
original_results = load_results('/content/results/stopping_experiment')

# Also try Drive backups
if not original_results:
    original_results = load_results('/content/drive/MyDrive/gradient_asymmetry_results')

if original_results and layernorm_results:
    print("COMPARISON: Original (No LayerNorm) vs LayerNorm Actor")
    print("="*85)
    print(f"{'Actor LR':<10} {'Critic LR':<10} {'Original':<15} {'LayerNorm':<15} {'Difference':<15}")
    print("-"*85)
    
    for orig in sorted(original_results, key=lambda x: (-x['actor_lr'], -x['critic_lr'])):
        ln = next((r for r in layernorm_results 
                   if r['actor_lr'] == orig['actor_lr'] and r['critic_lr'] == orig['critic_lr']), None)
        if ln:
            diff = ln['stopping_episode'] - orig['stopping_episode']
            diff_str = f"+{diff}" if diff > 0 else str(diff)
            improved = "IMPROVED" if diff > 100 else ""
            print(f"{orig['actor_lr']:<10} {orig['critic_lr']:<10} {orig['stopping_episode']:<15} {ln['stopping_episode']:<15} {diff_str:<10} {improved}")
    print("="*85)
elif layernorm_results:
    print("Original experiment results not found for comparison.")
else:
    print("No LayerNorm results found yet. Run the experiment first.")

## Cell 9: Analyze Pre-activation Statistics

In [ ]:
import json
import os
import numpy as np

results_dir = '/content/results/layernorm_experiment'

if os.path.exists(results_dir):
    print("Pre-activation Statistics (should be bounded with LayerNorm):")
    print("="*80)
    print(f"{'Actor LR':<10} {'Critic LR':<10} {'Min Preact':<15} {'Max Preact':<15} {'Saturation':<15}")
    print("-"*80)
    
    for exp_dir in sorted(os.listdir(results_dir)):
        tracking_file = os.path.join(results_dir, exp_dir, 'tracking_data.json')
        result_file = os.path.join(results_dir, exp_dir, 'results.json')
        if os.path.exists(tracking_file) and os.path.exists(result_file):
            with open(tracking_file) as f:
                tracking = json.load(f)
            with open(result_file) as f:
                result = json.load(f)
            
            act_hist = tracking.get('activation_history', [])
            if act_hist:
                last = act_hist[-1]
                print(f"{result['actor_lr']:<10} {result['critic_lr']:<10} {last['min_preact']:<15.2f} {last['max_preact']:<15.2f} {last['avg_actor_output_saturation']:<15.2%}")
    print("="*80)
    print("\nNote: With LayerNorm, pre-activations should stay bounded (~-3 to +3)")
    print("Without LayerNorm, high LR experiments showed values in the MILLIONS!")
else:
    print("No results found yet.")

## Cell 10: Download Results

In [ ]:
import shutil
import os

results_dir = '/content/results/layernorm_experiment'
output_zip = '/content/layernorm_results.zip'

if os.path.exists(results_dir):
    shutil.make_archive('/content/layernorm_results', 'zip', results_dir)
    print(f"Created: {output_zip}")
    print(f"Size: {os.path.getsize(output_zip) / 1024:.1f} KB")
    
    # Download
    from google.colab import files
    files.download(output_zip)
else:
    print("No results directory found.")